In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [160]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e713e4258c5b42f3a00b89fcaa820e9b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='7MQvdbzKXZRO5GYyWITSdxd0vvKJFjQg9VvK2Dxmo4xS',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_e713e4258c5b42f3a00b89fcaa820e9b.get_object(Bucket='windoutput-donotdelete-pr-dxhrchiibgtonl',Key='windoutput.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_excel(body)
data.head()


,TimeStamp,TotalPower[kW],TotalWindSpeed[m/s],TotalTemperature[°C],TotalWindDirection[°]
0,2016-01-01 00:00:00,1372.666667,5.936364,-4.060606,32.151515
1,2016-01-01 01:00:00,664.116667,4.743939,-4.045455,26.448485
2,2016-01-01 02:00:00,13.583333,2.871212,-3.757576,67.734848
3,2016-01-01 03:00:00,176.716667,3.675758,-3.939394,60.374242
4,2016-01-01 04:00:00,64.116667,3.280303,-3.863636,69.712121


In [161]:
import keras

In [162]:
from keras.models import Sequential 
from keras.layers import Dense 

In [163]:
model=Sequential()

In [164]:
model.add(Dense(input_dim=3,init="random_uniform",activation="relu",output_dim=3)) #no. of independent layers for independent layer(5)

In [165]:
model.add(Dense(output_dim=3,init="random_uniform",activation="relu"))

In [166]:
model.add(Dense(output_dim=1,init="random_uniform"))

In [167]:
model.compile(optimizer="adam", loss="mse", metrics=['mse']) #batch, mean sqaure, mean square

In [177]:
from sklearn.model_selection import train_test_split
X=data.drop(['TotalPower[kW]','TimeStamp'],axis=1)
y=data['TotalPower[kW]']
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=0)
print(X_train.head())

       TotalWindSpeed[m/s]  TotalTemperature[°C]  TotalWindDirection[°]
15187             6.278788             16.439394             168.350000
2575              1.660606             18.893939              36.165152
852               9.612121              1.712121              54.990909
4855              6.648485             17.742424             138.701515
8157              4.795455              4.909091             239.651515


In [180]:
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
14364/14364 [==============================] - 161s 11ms/step - loss: 3250979.9729 - mean_squared_error: 3250979.9729
Epoch 2/50
14364/14364 [==============================] - 158s 11ms/step - loss: 2865499.2074 - mean_squared_error: 2865499.2074
Epoch 3/50
14364/14364 [==============================] - 160s 11ms/step - loss: 2509389.2812 - mean_squared_error: 2509389.2812
Epoch 4/50
14364/14364 [==============================] - 160s 11ms/step - loss: 2191786.5100 - mean_squared_error: 2191786.5100
Epoch 5/50
14364/14364 [==============================] - 156s 11ms/step - loss: 1914923.9714 - mean_squared_error: 1914923.9714
Epoch 6/50
14364/14364 [==============================] - 158s 11ms/step - loss: 1682071.3738 - mean_squared_error: 1682071.3738
Epoch 7/50
14364/14364 [==============================] - 163s 11ms/step - loss: 1494232.5439 - mean_squared_error: 1494232.5439
Epoch 8/50
14364/14364 [==============================] - 158s 11ms/step - loss: 1345054.5677 - m

In [181]:
y_predict = model.predict(X_test)

In [182]:
import math
from sklearn.metrics import mean_squared_error
rmse = math.sqrt(mean_squared_error(y_test, y_predict))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 925.469


In [183]:
y_train_predict = model.predict(X_train)
rmse = math.sqrt(mean_squared_error(y_train, y_train_predict))
print('Train RMSE: %.3f' % rmse)

Train RMSE: 931.793


In [204]:
model.save("WindNN.h5")

In [205]:
!tar -zcvf WindNN.tgz WindNN.h5

WindNN.h5


In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [5]:
wml_credentials = {
  "iam_apikey_description": "Auto-generated for key 788c5651-0531-48b2-ab4b-8643a35563b9",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/967ff6f75e634ec5b6a837111b5df8d6::serviceid:ServiceId-b3e64b62-aa71-4bd6-804a-75097fd3aadd",
 "url": "https://us-south.ml.cloud.ibm.com"
}

In [6]:
client = WatsonMachineLearningAPIClient( wml_credentials )

In [7]:
client.deployments.list()

------------------------------------  ----------  ------  --------------  ------------------------  ---------------  -------------
GUID                                  NAME        TYPE    STATE           CREATED                   FRAMEWORK        ARTIFACT TYPE
abaea2f2-709c-4f26-b2fe-e03119b71666  ANN Model   online  DEPLOY_FAILURE  2020-07-11T03:08:41.253Z  tensorflow-1.15  model
794356ec-8820-4b64-adf6-08967ffdc9cd  ANN Model   online  DEPLOY_FAILURE  2020-07-11T02:59:15.112Z  tensorflow-1.15  model
895892a6-d791-4510-99a4-c4a509ce8a9b  LSTM Model  online  DEPLOY_FAILURE  2020-07-11T01:58:39.818Z  tensorflow-1.15  model
76008526-1bbb-4be7-92d8-12ca72046f44  LSTM Model  online  DEPLOY_SUCCESS  2020-07-10T07:07:00.637Z  tensorflow-1.15  model
------------------------------------  ----------  ------  --------------  ------------------------  ---------------  -------------


In [10]:
client.deployments.delete('794356ec-8820-4b64-adf6-08967ffdc9cd')

'SUCCESS'

In [209]:
metadata = {
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{'name':'keras', 'version': '2.2.4'}],
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.15",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.AUTHOR_NAME: "Kapil Gautam", 
    client.repository.ModelMetaNames.AUTHOR_EMAIL: "gautime.idk@gmail.com", 
    client.repository.ModelMetaNames.NAME: "WindEnergyOutput"
}

In [210]:
published_model = client.repository.store_model(model="WindNN.tgz", meta_props=metadata)

In [211]:
published_model

{'metadata': {'guid': '7cf2ff7e-4f91-41e6-8e17-c57eeb12a8a9',
  'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/fb623f56-308d-4a68-ba08-531736ca7fc7/published_models/7cf2ff7e-4f91-41e6-8e17-c57eeb12a8a9',
  'created_at': '2020-07-10T07:06:39.462Z',
  'modified_at': '2020-07-10T07:06:39.525Z'},
 'entity': {'runtime_environment': 'None Provided',
  'learning_configuration_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/fb623f56-308d-4a68-ba08-531736ca7fc7/published_models/7cf2ff7e-4f91-41e6-8e17-c57eeb12a8a9/learning_configuration',
  'author': {'name': 'Kapil Gautam'},
  'name': 'WindEnergyOutput',
  'learning_iterations_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/fb623f56-308d-4a68-ba08-531736ca7fc7/published_models/7cf2ff7e-4f91-41e6-8e17-c57eeb12a8a9/learning_iterations',
  'feedback_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/fb623f56-308d-4a68-ba08-531736ca7fc7/published_models/7cf2ff7e-4f91-41e6-8e17-c57eeb12a8a9/feedback',
  'lates

In [212]:
model_id = published_model["metadata"]["guid"]
model_id

'7cf2ff7e-4f91-41e6-8e17-c57eeb12a8a9'

In [213]:
model_deployment_details = client.deployments.create( artifact_uid=model_id, name="LSTM Model" )



#######################################################################################

Synchronous deployment creation for uid: '7cf2ff7e-4f91-41e6-8e17-c57eeb12a8a9' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='76008526-1bbb-4be7-92d8-12ca72046f44'
------------------------------------------------------------------------------------------------




In [5]:
client.deployments.list()

------------------------------------  ----------  ------  --------------  ------------------------  ---------------  -------------
GUID                                  NAME        TYPE    STATE           CREATED                   FRAMEWORK        ARTIFACT TYPE
76008526-1bbb-4be7-92d8-12ca72046f44  LSTM Model  online  DEPLOY_SUCCESS  2020-07-10T07:07:00.637Z  tensorflow-1.15  model
------------------------------------  ----------  ------  --------------  ------------------------  ---------------  -------------


In [219]:
scoring_endpoint = client.deployments.get_scoring_url(model_deployment_details)

In [220]:
scoring_endpoint

'https://us-south.ml.cloud.ibm.com/v3/wml_instances/fb623f56-308d-4a68-ba08-531736ca7fc7/deployments/76008526-1bbb-4be7-92d8-12ca72046f44/online'

In [221]:
model_deployment_details

{'metadata': {'guid': '76008526-1bbb-4be7-92d8-12ca72046f44',
  'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/fb623f56-308d-4a68-ba08-531736ca7fc7/deployments/76008526-1bbb-4be7-92d8-12ca72046f44',
  'created_at': '2020-07-10T07:07:00.637Z',
  'modified_at': '2020-07-10T07:07:00.883Z'},
 'entity': {'runtime_environment': 'None Provided',
  'name': 'LSTM Model',
  'scoring_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/fb623f56-308d-4a68-ba08-531736ca7fc7/deployments/76008526-1bbb-4be7-92d8-12ca72046f44/online',
  'deployable_asset': {'name': 'WindEnergyOutput',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/fb623f56-308d-4a68-ba08-531736ca7fc7/published_models/7cf2ff7e-4f91-41e6-8e17-c57eeb12a8a9',
   'guid': '7cf2ff7e-4f91-41e6-8e17-c57eeb12a8a9',
   'created_at': '2020-07-10T07:07:00.540Z',
   'type': 'model'},
  'description': 'Description of deployment',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'model_type': 'tensorflow-1.15',
  'st